<a href='https://colab.research.google.com/github/rabbitmetrics/langchain-13-min/blob/main/notebooks/langchain-13-min.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

### Read in pinecone vector database

In [1]:
# Load environment variables
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())


True

In [ ]:
# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('PINECONE_ENVIRONMENT'))
# print(os.getenv('PINECONE_API_KEY'))

In [3]:
# Read existing vector index from pinecone
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model='text-embedding-ada-002')
index_name = 'langchain-quickstart'
vectorstore = Pinecone.from_existing_index(index_name,embeddings_model)

c:\Python311\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Query vector database

In [4]:
# query = 'What types of lubricants are to be avoided for mechanisms design?'
# query = 'What are examples of harmonic drive gearboxes for aerospace applications?'
# query = 'What types of deployable decelerators are there'
# query = 'What can you tell me about the Orion Side Hatch Design? Please explain any failures and lessons learned in detail'
query = 'What can you tell me about latch mechanism design? Please explain any failures and lessons learned in detail with specific examples.'
# query = 'What can you tell me about ball-lock mechanism failures? Refer to specific examples.'

docs = vectorstore.similarity_search(query,k=4)
# docs_score = vectorstore.similarity_search_with_relevance_scores(query,k=4)

In [5]:
# Here's an example of the first document that was returned
print(docs[0].page_content[:450])

testing milestones, t he initial analysis of the hatch design was limited to standard FEM/stress modeling and 
rigid-body kinematic studies of the latch trains which all showed positive results with significant margins. 
There was a plan to do a mor e in-depth flex -body analysis to examine the complete interactions of the 
structure and latch trains, h owever this couldn’t be completed  in time. The CDR was held successfully (with 
many independ


### Query those docs to get an answer back

In [6]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [7]:
llm = OpenAI(temperature=1)
chain = load_qa_chain(llm, chain_type="stuff", verbose=True)

In [8]:
chain.run(input_documents=docs, question=query)

" In designing the latch mechanisms, the initial analysis was limited to standard FEM/stress modeling and rigid-body kinematic studies. There was a plan for a  more in-depth flex -body analysis, however this wasn't completed in time and the team proceeded into development testing. During development testing, numerous structural and life tests were conducted on the latch mechanisms. One lesson learned was that spring force hysteresis needs to be taken into consideration for complex mechanical systems, as spring force can drop by 10%-20% due to interactions with other parts of the system. A specific example of a latch mechanism failure investigation was that of a static load test of a single latch. After the failure, a fishbone diagram was constructed which highlighted more than 50 possible root cause contributors and further analysis was conducted. The originally planned flex -body analysis was given new urgency, and new software models and tools were used to analyze the cause of the fa

In [9]:
# Add something with a system description so the chatbot knows some context.
# Try using the multi-context question tool as an input to a chain query.
# Use cosine scores to find more docs which are relevant.
# Try to create an agent to do a more extensive search.

### Stuff to try

In [ ]:
# TODO: Check this out: https://python.langchain.com/docs/modules/data_connection/retrievers/ 

### Fancy stuff that is kinda sorta working

In [10]:
# Retrieval
# https://python.langchain.com/docs/use_cases/question_answering/

# simple
# docs = vectorstore.similarity_search(query)
# len(docs)

# deeper, using multiple question formulations to get relevant data
# This cool but does not work that well with how it's integrated...
# import logging
# from langchain.chat_models import ChatOpenAI
# from langchain.retrievers.multi_query import MultiQueryRetriever


# logging.basicConfig()
# logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

# llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
# retriever_from_llm = MultiQueryRetriever.from_llm(llm=llm,retriever=vectorstore.as_retriever(),parser_key='lines')


# # print(retriever_from_llm.dict())
# unique_docs = retriever_from_llm.get_relevant_documents(query=query)
# len(unique_docs)

In [11]:
# Generate
# https://python.langchain.com/docs/use_cases/question_answering/

# Return source docs
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=.5)
qa_chain = RetrievalQA.from_llm(llm,retriever=vectorstore.as_retriever(),
                                       return_source_documents=True)

results = qa_chain({'query': query})

In [12]:
print(results['query']+'\n')
print(results['result']+'\n')

print('Sources:')
for doc in results['source_documents']:
    print(doc.metadata)

What can you tell me about latch mechanism design? Please explain any failures and lessons learned in detail with specific examples.

The latch mechanism design for the Orion Side Hatch underwent testing and development, with several lessons learned and failures encountered. 

One lesson learned was the importance of considering spring hysteresis in the design. Spring hysteresis refers to the loss of spring force after cycling. The development testing confirmed that the spring force dropped by 10% - 20% after cycling. This hysteresis loss was compensated for in Moog's new latch system, which accounted for all environmental conditions and provided repeatable performance. The lesson here is that initial design development should include the consideration of spring force hysteresis losses.

Another lesson learned was the need for accurate modeling and analysis of complex latch mechanisms with many parts. The initial analysis of the latch design was limited to standard FEM/stress modeling 

In [13]:
# Lots of different ways to do this under https://python.langchain.com/docs/use_cases/question_answering/
from langchain.chains import RetrievalQAWithSourcesChain

qa_chain = RetrievalQAWithSourcesChain.from_llm(llm,retriever=vectorstore.as_retriever())

result = qa_chain({"question": query})
result

{'question': 'What can you tell me about latch mechanism design? Please explain any failures and lessons learned in detail with specific examples.',
 'answer': 'There is no specific information provided about latch mechanism failures or lessons learned in the given text.',
 'sources': ''}